In [1]:
import sys
from tqdm import tqdm
import os
import json
import time
import argparse

import pandas as pd
import numpy as np

from Bio import SeqIO
import esm

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset

from torchinfo import summary

from sklearn.metrics import f1_score, roc_auc_score, roc_curve, auc, precision_recall_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import itertools
import json

# Get the path of the Python script
sys.path.append('./../../../src/')

from utils import *
from utils_torch import * 
from MHCCBM import *
# from TAPPredictor import *
from TAPPredictor_CNN import *


seed = 42
set_seed(42)
print("Seed: ", seed)



Seed:  42


# create config json

In [2]:


# hidden_channels_ls = [[1024], [512], [256], [128]]
# epochs = 500
# batch_size_ls = [32,64]
# lr_ls = [1e-3, 1e-5]
# ksize_ls = [3, 5, 7]
# dropout_p_ls = [0, 0.1]

# for i, (hc, ks, bs, lr, dp) in enumerate(itertools.product(hidden_channels_ls, ksize_ls, batch_size_ls, lr_ls, dropout_p_ls)):
#     print(i, (hc, ks, bs, lr, dp))
    
#     config_dict = { "project" : "MHCCBM",
#                     "name" : "CNN_"+str(i), 
#                     "config" : {
#                         "hidden_channels" : hc,
#                         "epochs" : epochs,
#                         "kernel_size": ks,
#                         "pool_kernel_size":ks,
#                         "classes" : 2,
#                         "seed": 42,
#                         "batch_size" : bs,
#                         "lr" : lr,
#                         "dataset" : "X.pkl, y.pkl (embedded peptide seq from classification",
#                         "dropout_p" : dp,
#                         "architecture" : "CNN"}
#                     }
#     with open('./../config/hyperparameter_tuning/CNN/config_'+str(i)+'.json','w') as f:
#         json.dump(config_dict,f)

# config_dict = { "project" : "MHCCBM",
#                     "name" : "CNN_"+str(98), 
#                     "config" : {
#                         "hidden_channels" : [128,128,128],
#                         "epochs" : 500,
#                         "kernel_size": 5,
#                         "pool_kernel_size":5,
#                         "classes" : 2,
#                         "seed": 42,
#                         "batch_size" : 32,
#                         "lr" : 1e-05,
#                         "dataset" : "X.pkl, y.pkl (embedded peptide seq from classification",
#                         "dropout_p" : 0.1,
#                         "architecture" : "CNN_idx83_3layer"}
#                     }
# with open('./../config/hyperparameter_tuning/CNN/config_'+str(98)+'.json','w') as f:
#     json.dump(config_dict,f)

In [3]:
hidden_channels_ls = [[1024], [512], [256], [128]]
epochs = 500
batch_size_ls = [32,64]
lr_ls = [1e-3, 1e-5]
ksize_ls = [3, 5, 7]
dropout_p_ls = [0, 0.1]
combinations = [i for i in itertools.product(hidden_channels_ls, ksize_ls, batch_size_ls, lr_ls, dropout_p_ls)]
ncombinations = len(combinations)

In [4]:
result_df = []
for idx in range(ncombinations):

    result_ = pd.read_csv('./../results/hyperparameter_tuning_result/CNN/CNN_'+str(idx)+'.csv')
    result_['idx'] = [idx]*len(result_)
    result_df = result_df + [result_]
        
result_df = pd.concat(result_df)
result_df.groupby('idx').mean().sort_values('auroc_test')

,Unnamed: 0,time_elapsed,epoch,fold,f1_train,auroc_train,auprc_train,f1_test,auroc_test,auprc_test
idx,,,,,,,,,,
95,0.0,141.523261,NaN,2.0,0.784300,0.884698,0.947197,0.718470,0.786978,0.901351
94,0.0,142.821728,NaN,2.0,0.792072,0.884454,0.947069,0.729039,0.787947,0.898305
91,0.0,136.176186,NaN,2.0,0.828188,0.914901,0.961714,0.748200,0.801876,0.906727
71,0.0,267.768988,NaN,2.0,0.816420,0.907686,0.958294,0.752559,0.803021,0.906044
70,0.0,244.813128,NaN,2.0,0.814433,0.906606,0.957647,0.751019,0.804509,0.905368
...,...,...,...,...,...,...,...,...,...,...
0,0.0,355.384811,59.0,2.0,1.000000,1.000000,1.000000,0.773646,0.833245,0.919326
73,0.0,42.599087,60.6,2.0,1.000000,1.000000,1.000000,0.785003,0.833389,0.920929
49,0.0,81.228782,60.4,2.0,1.000000,1.000000,1.000000,0.781607,0.834107,0.920187


In [7]:
combinations[29], combinations[72], combinations[58]

(([512], 3, 64, 0.001, 0.1),
 ([128], 3, 32, 0.001, 0),
 ([256], 5, 32, 1e-05, 0))

In [6]:
pd.read_csv('./../results/hyperparameter_tuning_result/CNN/CNN_'+str(97)+'.csv').mean()

Unnamed: 0        0.000000
time_elapsed    186.863738
epoch           375.250000
fold              2.000000
f1_train          0.960658
auroc_train       0.996254
auprc_train       0.998302
f1_test           0.753294
auroc_test        0.812790
auprc_test        0.910546
dtype: float64